In [411]:
# Importing libraries neeeded for the project
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [412]:
#!/usr/bin/env /Applications/MAMP/Library/bin/python

import mysql.connector

config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'ecotourisme',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cursor = cnx.cursor(dictionary=True)

cursor.execute('SELECT * FROM `ecologie` where id_pays = "FR" LIMIT 15')
results = cursor.fetchall()
cnx.close()

data1 = pd.DataFrame(results)

data1.head()

# afficher data1 par ordre decroissant annee
data1 = data1.sort_values(by='annee', ascending=True) 
data1.head()

,id,id_pays,annee,co2,ges,elecRenew,GES_hab
0,63,FR,1990,356297.9,496236.8940,13.370086,8.81505
1,274,FR,1991,380691.3,521271.9651,13.221293,9.25977
2,485,FR,1992,368655.7,506713.6172,15.455850,9.00116
3,696,FR,1993,349524.6,487761.4056,14.235830,8.66450
4,907,FR,1994,344475.4,482039.2237,17.056469,8.56285


In [413]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

# Supprimer les valeurs manquantes du DataFrame
data1 = data1.dropna()

# Sélection des variables explicatives et de la variable à prédire
X = data1[[ 'GES_hab', 'elecRenew']]
y = data1['elecRenew']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle de régression linéaire
model = LinearRegression()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer l'erreur quadratique moyenne (RMSE) pour évaluer les performances du modèle
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

# Afficher les coefficients de régression
print("Coefficients:", model.coef_)

# Afficher l'interception
print("Intercept:", model.intercept_)
    
# Générer les années
num_predictions = 15
years = range(2021, 2021 + num_predictions)

# Générer les valeurs de GES_hab et elecRenew a l'aide du modele 
GES_hab_values = [model.coef_[0] * x + model.intercept_ for x in data1['GES_hab']]
elecRenew_values = [model.coef_[1] * x + model.intercept_ for x in data1['elecRenew']]

# Créer un DataFrame
future_data = pd.DataFrame({
    'annee': years,
    'GES_hab': GES_hab_values,
    'elecRenew': elecRenew_values
})

# Sélectionner les colonnes 'GES_hab' et 'elecRenew'
future_X = future_data[['GES_hab', 'elecRenew']]

# Faire des prédictions pour les années à venir
future_y_pred = model.predict(future_X)

# Créer un DataFrame pour les prédictions
forecast = pd.DataFrame({
    'annee': years,
    'GES_hab': future_y_pred[0],
    'elecRenew': future_y_pred[1]
})

# Afficher les prédictions
forecast


co2 = data1[['annee', 'GES_hab', 'elecRenew']]
co2 = pd.concat([co2, forecast], ignore_index=True)

co2.head()



RMSE: 1.0255800994045674e-15
Coefficients: [-4.7430543e-15  1.0000000e+00]
Intercept: 5.3290705182007514e-14


,annee,GES_hab,elecRenew
0,1990,8.81505,13.370086
1,1991,9.25977,13.221293
2,1992,9.00116,15.455850
3,1993,8.66450,14.235830
4,1994,8.56285,17.056469


In [414]:
# Créer un graphique à barres pour les prédictions
fig = px.bar(co2, x='annee', y='GES_hab', title='Prédictions de la part GES_hab dans la production d\'électricité aux États-Unis')
fig.update_layout(xaxis_title='Année', yaxis_title='Pourcentage')
fig.show()

Modele 1 : ARIMA (bof) 

In [415]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
# Utiliser vos données déjà extraites et normalisées
data = data1.set_index('annee')

# Fractionnement des données en ensembles d'entraînement et de test
test_data = data[data.index <= 2000]  # Données jusqu'en 2020 pour l'entraînement
train_data = data[data.index > 1990]    # Données à partir de 2021 pour les tests

# Réinitialiser l'index
train_data_reset = train_data.reset_index()
test_data_reset = test_data.reset_index()

# Entraînement du modèle ARIMA
model = ARIMA(train_data_reset['co2'], order=(5,1,0))
model_fit = model.fit()

# Réduire le nombre de pas de prédiction
forecast_steps = min(len(test_data_reset), 10)  # Par exemple, prédire 10 pas à l'avance maximum

# Prédictions futures
forecast = model_fit.forecast(steps=forecast_steps)

# Affichage des prédictions
print(forecast)

14    377673.098317
15    377602.063471
16    377461.023975
17    377740.185798
18    377780.099536
19    377752.542219
20    377742.125274
21    377742.430628
22    377756.158708
23    377756.946477
Name: predicted_mean, dtype: float64


In [416]:
# Définir le nombre de prédictions
num_predictions = len(forecast)

# Générer les années
years = range(2021, 2021 + num_predictions)

# Créer un DataFrame avec les prédictions et les années
predictions_df = pd.DataFrame({
    'annee': years,
    'co2': forecast
})


co2 = data1[['annee', 'co2']]
co2 = pd.concat([co2, predictions_df], ignore_index=True)

co2.head()


,annee,co2
0,1990,356297.9
1,1991,380691.3
2,1992,368655.7
3,1993,349524.6
4,1994,344475.4


In [417]:
# Ajouter data à data1
data1 = pd.concat([data1, predictions_df])

# Afficher les prédictions
fig = px.line(data1, x='annee', y='co2', title='Prédictions de la consommation d\'électricité renouvelable')
fig.show()
